In [1]:
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
import numpy as np
from typing import List
from functools import partial
from operator import itemgetter
from itertools import combinations
from typing import List, Tuple
import random

In [8]:
class Move(Enum):
    '''
    Selects where you want to place the taken piece. The rest of the pieces are shifted
    '''
    TOP = 0
    BOTTOM = 1
    LEFT = 2
    RIGHT = 3


class Player(ABC):
    def __init__(self, genes=None) -> None:
        '''You can change this for your player if you need to handle state/have memory'''
        self.genes = genes or self.generate_genes()

    def generate_genes(self):
        '''Generate random genes for the player'''
        return [random.choice(list(Move)) for _ in range(100)]  # Adjust the size as needed

    @abstractmethod
    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        '''
        The game accepts coordinates of the type (X, Y). X goes from left to right, while Y goes from top to bottom, as in 2D graphics.
        Thus, the coordinates that this method returns shall be in the (X, Y) format.

        game: the Quixo game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions and a move among TOP, BOTTOM, LEFT and RIGHT
        '''
        pass

class GeneticPlayer(Player):
    def __init__(self, population_size, generations,lamda,id_player): ###Inizializzazione degli elementi della classe genetica
        super().__init__()
        self.population_size = population_size
        self.generations = generations
        self.population = self.initialize_population()
        self.lamda = lamda
        self.id=id_player

    def mossa_accettabile(self,from_pos: tuple[int,int]):
       # acceptable only if in border
        acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < 5)
            # check if it is in the last row
            or (from_pos[0] == 4 and from_pos[1] < 5)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < 5)
            # check if it is in the last column
            or (from_pos[1] == 4 and from_pos[0] < 5)
            # and check if the piece can be moved by the current player
        )
        return acceptable

    def check_condition(self,matrix, row, col):
    # Conta gli uni nella riga e nella colonna di riferimento oppure gli zeri in base all'id
      if self.id==0:
        count_row = np.count_nonzero(matrix[row, :] == 1) ##conto uni riga e colonna
        count_col = np.count_nonzero(matrix[:, col] == 1)
        count_diag_upper = np.count_nonzero(np.diagonal(matrix, offset=col - row) == 1) ##conto uni diagonale superiore
        count_diag_lower = np.count_nonzero(np.diagonal(np.flip(matrix, axis=0), offset=row - (matrix.shape[1] - 1 - col)) == 1)
      elif self.id==1:
        count_row = 5-np.count_nonzero(matrix[row, :] == 1) ##conto zeri
        count_col = 5-np.count_nonzero(matrix[:, col] == 1)
        count_diag_upper=5-np.count_nonzero(np.diagonal(matrix, offset=col - row) == 1)
        count_diag_lower =5-np.count_nonzero(np.diagonal(np.flip(matrix, axis=0), offset=row - (matrix.shape[1] - 1 - col)) == 1)
    # Verifica la condizione
      return count_row < 4 and count_col < 4 and count_diag_upper < 4 and count_diag_lower < 4

    def check_condition_zero(self,matrice):
    #  Conta le sequenze di zeri in ogni riga e anche per gli uni
     if self.id==0:
       zero_sequences_row = np.array([np.count_nonzero(np.diff(np.where(row == 0)) == 1) for row in matrice])
       zero_sequences_col = np.array([np.count_nonzero(np.diff(np.where(col == 0)) == 1) for col in matrice.T])
       condizione_righe = np.sum(zero_sequences_row >= 3) > 0
       condizione_colonne = np.sum(zero_sequences_col >= 3) > 0
     elif self.id==1:
       uno_sequences_row = np.array([np.count_nonzero(np.diff(np.where(row == 1)) == 1) for row in matrice])
       uno_sequences_col = np.array([np.count_nonzero(np.diff(np.where(col == 1)) == 1) for col in matrice.T])
       condizione_righe = np.sum(uno_sequences_row >= 3) > 0
       condizione_colonne = np.sum(uno_sequences_col >= 3) > 0

    # Verifica la condizione
     return (condizione_righe and condizione_colonne)

    def verifica_sottomatrice(self,matrice):
    # Verifica che la matrice sia almeno 5x5
      if matrice.shape != (5, 5):
        raise ValueError("La matrice deve essere 5x5")
    # Estrai la sottomatrice 3x3 iniziando dalla riga 1 e colonna 1
      sottomatrice = matrice[1:4, 1:4]
    # Conta il numero di elementi nella sottomatrice che sono uguali a 1
      numero_di_uno = np.sum(sottomatrice == 1)
      numero_di_zeri=np.sum(sottomatrice==0)
    # Verifica se il numero di 1 è minore di 5
      if self.id==0:
        return numero_di_uno < 3

      return numero_di_zeri < 3

    def initialize_population(self):##INIZIALIZZAZIONE POPOLAZIONE DI SOLE MOSSE IN POSIZIONI ACCETTABILI
          population = []
          for _ in range(self.population_size):
            strategy = [(random.choice(list(Move)), (random.randint(0, 4), random.randint(0, 4))) for _ in range(30)]
            for i,elemento in enumerate(strategy):
              if self.mossa_accettabile(elemento[1])==False:
                strategy.pop(i)
                pass
            population.append(strategy)
          return population

    def tournament_selection(self, candidates,game):
          gamecopy=deepcopy(game)
          tournament_size =3 # Puoi regolare questo valore in base alle tue esigenze
          selected_candidates = random.sample(candidates, tournament_size)
          best_candidate = max(selected_candidates, key=lambda x: self.evaluate_fitness(x, gamecopy))
          best_candidate_index = candidates.index(best_candidate)
          return best_candidate_index

    def evaluate_fitness(self, strategy, game):
        total_fitness = 0
        gamecopy = deepcopy(game)
        current_player_idx = gamecopy.current_player_idx
        for i,move in enumerate(strategy):
              ok = gamecopy._Game__move(move[1], move[0], current_player_idx)
              if not ok:
                  total_fitness -=10  # Penalize for invalid moves
              winner = gamecopy.check_winner()
              if winner == 1-self.id:
                  #strategy.pop(i)
                  total_fitness -= 20000  # Strong penalty for benefiting the opponent
              elif winner == self.id: ##Cioè se vince il nostro giocatore che sarebbe quello Genetico
                 #ok=gamecopy._Game__move(move[1], move[0], current_player_idx)
                 total_fitness += 10000000000000 # Reward for winning
                 break
              elif winner ==-1:
                 matrice=gamecopy.get_board()
                 if not self.verifica_sottomatrice(matrice):
                       total_fitness-=5000
                 if not self.check_condition_zero(matrice):
                       total_fitness-=3000
        ##Fitness totale
        average_fitness = total_fitness
        return average_fitness

    def make_move(self, game):
        offspring = []
        gamecopy = deepcopy(game)
        parents_mutated = []
        ok=False
        lista_finale_strategie=[]
        lista_combinata_fitness_popolazione=[]
        lista_best_strategy=[]
        lista_ulteriore=[]
        parents=[]

        # DOBBIAMO QUA FAR ANDARE AVANTI IL NOSTRO METODO PER UN NUMERO X=? DI GENERAZIONI
        for _ in range(self.generations):
            lista = self.initialize_population()
            lista.sort(key=lambda x: self.evaluate_fitness(x,game),reverse=True)
            # Tournament selection---> applicandola il codice sta diverso tempo a runnare ma arriva a risultato
            for _ in range(len(lista)):
                 tournament_candidates = random.sample(lista,5)
                 best_strategy_index = self.tournament_selection(tournament_candidates,game)
                 parent = tournament_candidates[best_strategy_index]
                 parents.append(parent)
            #parents1 = lista[0]
            #parents = [parents1]


            # MUTATION vs CROSSOVER STRATEGY---->Strategy 2 vista a lezione nel gruppo di slide della Genetic Programming

            #Mutation of elements of parents
            if random.random()<0.4:
             for i in range(len(parents)):
                element = parents[i]  #lista interna visto che io considero una lista di liste di azioni che faccio nella board
                mutation_type =random.choice(["replace_move","replace_position","replace_all"])
                if mutation_type == "replace_move":
                    #Replace a random move in the strategy with a new random move
                    random_index = random.randint(0, len(element) - 1)
                    element[random_index] = (random.choice(list(Move)), element[random_index][1])
                elif mutation_type=='replace_position':
                    random_index = random.randint(0, len(element) - 1)
                    element[random_index] = (element[random_index][0], (random.randint(0, 4), random.randint(0, 4)))
                elif mutation_type=='replace_all':
                   random_index = random.randint(0, len(element) - 1)
                   element[random_index] = (random.choice(list(Move)), (random.randint(0, 4), random.randint(0, 4)))
                parents_mutated.append(element)
             offspring=parents_mutated
            else:
              #Crossover di tipo one cut crossover
              parents_mutated=parents
              while len(offspring) < len(self.population):
                 parent1,parent2 = random.choices(parents_mutated, k=2)
                 crossover_index = random.randint(1, min(len(parent1), len(parent2)) - 1)
                 child = parent1[:crossover_index] + parent2[crossover_index:]
                 offspring.append(child)

            self.population = offspring ## la nuova popolazione composta dai figli

            ###Incremento il numero di individui nella poplazione di un fattore lambda
            for _ in range(self.lamda):
               strategy = [(random.choice(list(Move)), (random.randint(0, 4), random.randint(0, 4))) for _ in range(30)]
               self.population.append(strategy)

            fitness_scores = [self.evaluate_fitness(strategy, gamecopy) for strategy in self.population]
            for i,strategia in enumerate(self.population):
                lista_combinata_fitness_popolazione.append((fitness_scores[i],strategia))

        lista_combinata_fitness_popolazione.sort(key=lambda x:x[0],reverse=True)
        i=0
        while ok==False and i<=(len(lista_combinata_fitness_popolazione[0][1])-1):
           # Scegli la mossa dalla strategia migliore
           move, position = lista_combinata_fitness_popolazione[0][1][i]
           i=i+1
           # Applica la mossa allo stato attuale del tavolo se è tutto ok
           ok = gamecopy._Game__move(position, move, gamecopy.current_player_idx)
           if gamecopy.check_winner()==self.id:
             return position,move
        if ok==False:
           while ok==False:
             move,position=(random.choice(list(Move)), (random.randint(0, 4), random.randint(0, 4)))
             ok=gamecopy._Game__move(position, move, gamecopy.current_player_idx)
             if gamecopy.check_winner()==self.id:
                return position,move
        return position, move

In [9]:
class Game(object):
    def __init__(self) -> None:
        self._board = np.ones((5, 5), dtype=np.uint8) * -1
        self.current_player_idx = 1

    def get_board(self) -> np.ndarray:
        '''
        Returns the board
        '''
        return deepcopy(self._board)

    def get_current_player(self) -> int:
        '''
        Returns the current player
        '''
        return deepcopy(self.current_player_idx)

    def print(self):
        '''Prints the board. -1 are neutral pieces, 0 are pieces of player 0, 1 pieces of player 1'''
        print(self._board)

    def check_winner(self) -> int:
        '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
        # for each row
        for x in range(self._board.shape[0]):
            # if a player has completed an entire row
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]):
                # return the relative id
                return self._board[x, 0]
        # for each column
        for y in range(self._board.shape[1]):
            # if a player has completed an entire column
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]):
                # return the relative id
                return self._board[0, y]
        # if a player has completed the principal diagonal
        if self._board[0, 0] != -1 and all(
            [self._board[x, x]
                for x in range(self._board.shape[0])] == self._board[0, 0]
        ):
            # return the relative id
            return self._board[0, 0]
        # if a player has completed the secondary diagonal
        if self._board[0, -1] != -1 and all(
            [self._board[x, -(x + 1)]
             for x in range(self._board.shape[0])] == self._board[0, -1]
        ):
            # return the relative id
            return self._board[0, -1]
        return -1

    def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        winner = -1
        n_mosse=0
        while winner < 0:
            self.current_player_idx += 1
            self.current_player_idx %= len(players)
            ok = False
            while not ok:
                from_pos, slide = players[self.current_player_idx].make_move(
                    self)
                ok = self.__move(from_pos, slide, self.current_player_idx)
                if ok==True:
                  n_mosse+=1
                  print(f"\n Il giocatore {players[self.current_player_idx]} ha fatto la mossa {n_mosse}")
                  self.print()
            winner = self.check_winner()
        return winner

    def __move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        '''Perform a move'''
        if player_id > 2:
            return False
        # Oh God, Numpy arrays
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])
        acceptable = self.__take((from_pos[1], from_pos[0]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[1], from_pos[0]), slide)
            if not acceptable:
                self._board[(from_pos[1], from_pos[0])] = deepcopy(prev_value)
        return acceptable

    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        '''Take piece'''
        # acceptable only if in border
        acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < 5)
            # check if it is in the last row
            or (from_pos[0] == 4 and from_pos[1] < 5)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < 5)
            # check if it is in the last column
            or (from_pos[1] == 4 and from_pos[0] < 5)
            # and check if the piece can be moved by the current player
        ) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable:
            self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        '''Slide the other pieces'''
        # define the corners
        SIDES = [(0, 0), (0, 4), (4, 0), (4, 4)]
        # if the piece position is not in a corner
        if from_pos not in SIDES:
            # if it is at the TOP, it can be moved down, left or right
            acceptable_top: bool = from_pos[0] == 0 and (
                slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is at the BOTTOM, it can be moved up, left or right
            acceptable_bottom: bool = from_pos[0] == 4 and (
                slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is on the LEFT, it can be moved up, down or right
            acceptable_left: bool = from_pos[1] == 0 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT
            )
            # if it is on the RIGHT, it can be moved up, down or left
            acceptable_right: bool = from_pos[1] == 4 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT
            )
        # if the piece position is in a corner
        else:
            # if it is in the upper left corner, it can be moved to the right and down
            acceptable_top: bool = from_pos == (0, 0) and (
                slide == Move.BOTTOM or slide == Move.RIGHT)
            # if it is in the lower left corner, it can be moved to the right and up
            acceptable_left: bool = from_pos == (4, 0) and (
                slide == Move.TOP or slide == Move.RIGHT)
            # if it is in the upper right corner, it can be moved to the left and down
            acceptable_right: bool = from_pos == (0, 4) and (
                slide == Move.BOTTOM or slide == Move.LEFT)
            # if it is in the lower right corner, it can be moved to the left and up
            acceptable_bottom: bool = from_pos == (4, 4) and (
                slide == Move.TOP or slide == Move.LEFT)
        # check if the move is acceptable
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        # if it is
        if acceptable:
            # take the piece
            piece = self._board[from_pos]
            # if the player wants to slide it to the left
            if slide == Move.LEFT:
                # for each column starting from the column of the piece and moving to the left
                for i in range(from_pos[1], 0, -1):
                    # copy the value contained in the same row and the previous column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i - 1)]
                # move the piece to the left
                self._board[(from_pos[0], 0)] = piece
            # if the player wants to slide it to the right
            elif slide == Move.RIGHT:
                # for each column starting from the column of the piece and moving to the right
                for i in range(from_pos[1], self._board.shape[1] - 1, 1):
                    # copy the value contained in the same row and the following column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i + 1)]
                # move the piece to the right
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            # if the player wants to slide it upward
            elif slide == Move.TOP:
                # for each row starting from the row of the piece and going upward
                for i in range(from_pos[0], 0, -1):
                    # copy the value contained in the same column and the previous row
                    self._board[(i, from_pos[1])] = self._board[(
                        i - 1, from_pos[1])]
                # move the piece up
                self._board[(0, from_pos[1])] = piece
            # if the player wants to slide it downward
            elif slide == Move.BOTTOM:
                # for each row starting from the row of the piece and going downward
                for i in range(from_pos[0], self._board.shape[0] - 1, 1):
                    # copy the value contained in the same column and the following row
                    self._board[(i, from_pos[1])] = self._board[(
                        i + 1, from_pos[1])]
                # move the piece down
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable

In [10]:
import random
class RandomPlayer(Player):

    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move


class MyPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move

In [11]:
    vince0=0
    vince1=0
    i=0
    for _ in range(10):
     i=i+1
     g = Game() ##inizializzo il gioco
     print("Tabella iniziale")
     g.print() ##stampo la tabella iniziale per completezza
     player2 = GeneticPlayer(10,10,1,1) ##inizializzo il giocatore 1 con la strategia di Programmazione Genetica
     #player1=RandomPlayer()
     player1 = RandomPlayer() ##Uso un random player per vedere come procede la mia strategia
     winner = g.play(player1, player2) ##verifichiamo se abbiamo un vincitore
     print(f"Winner: Player {winner}") ##Se abbiamo un vincitore entro un certo range di mosse che ho impostato a 150 totali per evitare che il gioco vada in loop allora lo stampo
     if winner==0:
         vince0+=1
     else:
         vince1+=1
     print(f"Alla iterazione {i} siamo che il player {player1} è per ora {vince0} mentre il player {player2} per ora è a {vince1}")

Tabella iniziale
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]

 Il giocatore <__main__.RandomPlayer object at 0x7bfa7032e980> ha fatto la mossa 1
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1]]

 Il giocatore <__main__.GeneticPlayer object at 0x7bfa7032e740> ha fatto la mossa 2
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1]]

 Il giocatore <__main__.RandomPlayer object at 0x7bfa7032e980> ha fatto la mossa 3
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [ 0 -1 -1 -1  0]]

 Il giocatore <__main__.GeneticPlayer object at 0x7bfa7032e740> ha fatto la mossa 4
[[ 1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [ 0 -1 -1 -1  0]]

 Il giocatore <__main__.RandomPlayer object at 0x7bfa7032e980> ha fatto la mossa 5
[[ 1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1]
 [ 0 -1 -1 -1  0]]

 Il giocatore 